# Minería de Datos sobre Capa Oro

Este notebook desarrolla un análisis completo de **minería de datos** sobre información meteorológica de la provincia de Misiones.

### Objetivos
1. **Generar y explorar la variable `LLUEVE`** para identificar días lluviosos.
2. **Descubrir patrones y relaciones ocultas** mediante visualización, combinaciones y clustering.
3. **Cubrir las etapas de minería de datos** dentro del flujo **CRISP-DM**:
   - Comprensión de los datos
   - Preparación
   - Minería / Modelado
   - Evaluación

El enfoque es **no supervisado**, buscando patrones sin usar etiquetas predefinidas.

# Importar las librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from mlxtend.frequent_patterns import apriori, association_rules
import warnings

# Deshabilitar warnings futuros
warnings.simplefilter(action='ignore', category=FutureWarning)

MINERIA_DIR = Path('../data/mineria')
MINERIA_DIR.mkdir(parents=True, exist_ok=True)

print("Importación de librerías completada.")

## Carga de datos

Se cargan los datasets generados en la **Capa Oro**:
- `dataset_oro_diario.csv` → Datos diarios por estación.
- `dataset_oro_horario.csv` → Datos horarios por estación.

In [ ]:
# Carga de datasets Oro
file_diario = '../data/oro/dataset_oro_diario.csv'
file_horario = '../data/oro/dataset_oro_horario.csv'

df_diario = pd.read_csv(file_diario)
df_horario = pd.read_csv(file_horario)

print("Dimensiones datos diarios:", df_diario.shape)
print("Dimensiones datos horarios:", df_horario.shape)

df_diario.head()

## Generación de la variable `LLUEVE`

Como no contamos con precipitación directa, se define la lluvia mediante la regla heurística:

- **1 (Llueve)** si `Humedad > 75%` y `Presión < 1010 hPa`  
- **0 (No llueve)** en caso contrario

Esto permite contar con una etiqueta binaria para minería de datos.

In [ ]:
# Crear variables binarias de lluvia

# Condición: humedad > 75% y presión < 1010 hPa
humedad = 75
presion = 1010

df_diario['LLUEVE'] = ((df_diario['HUM_MEAN'] > humedad) & 
                       (df_diario['PNM_MEAN'] < presion)).astype(int)

df_horario['LLUEVE'] = ((df_horario['HUM'] > humedad) & 
                        (df_horario['PNM'] < presion)).astype(int)

print("Variables generadas")
print("")
print("Distribución LLUEVE (0=No,1=Sí):")
print("")
print(df_diario['LLUEVE'].value_counts())

df_diario[['ESTACION','FECHA','HUM_MEAN','PNM_MEAN','LLUEVE']].head(10)

# Guardar el dataset con las variables generadas

In [ ]:
# Guardar los datasets con la nueva variable. Exportación a CSV
df_diario.to_csv(MINERIA_DIR / 'dataset_mineria_diario.csv', index=False)
df_horario.to_csv(MINERIA_DIR / 'dataset_mineria_horario.csv', index=False)

print("Archivos generados con la variable LLUEVE.")
print(f"Filas diarias: {len(df_diario)} | Filas horarias: {len(df_horario)}")
print("Muestra de datos con la variable LLUEVE (diario):")
print(df_diario[['ESTACION','FECHA','HUM_MEAN','PNM_MEAN','LLUEVE']].head())

## Exploración y visualización de la variable `LLUEVE`

Se analiza la distribución de días con lluvia, por estación y de manera temporal.
Además, se incluye un **heatmap de correlación** para detectar **relaciones ocultas** entre variables.

In [ ]:
# Conteo lluvia
df_diario['LLUEVE'].value_counts().plot(kind='bar', color=['skyblue','blue'])
plt.title("Distribución de días con lluvia")
plt.xlabel("LLUEVE (0=No, 1=Sí)")
plt.ylabel("Cantidad de días")
plt.show()

# Lluvia por estación
df_diario.groupby('ESTACION')['LLUEVE'].sum().sort_values(ascending=False).plot(kind='bar', figsize=(10,5), color='green')
plt.title("Cantidad de días con lluvia por estación")
plt.xticks(rotation=45, ha='right')
plt.show()

### Heatmap de correlación general

In [ ]:

vars_corr = ['TEMP_MEAN','HUM_MEAN','PNM_MEAN','WIND_SPEED_MEAN','LLUEVE']

plt.figure(figsize=(8,6))
sns.heatmap(df_diario[vars_corr].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de correlación - Todas las estaciones")
plt.show()



### Generar un heatmap por cada estación

In [ ]:
for estacion in df_diario['ESTACION'].unique():
    df_est = df_diario[df_diario['ESTACION'] == estacion]
    
    plt.figure(figsize=(8,6))
    sns.heatmap(df_est[vars_corr].corr(), annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
    plt.title(f"Matriz de correlación - {estacion}")
    plt.show()


## Combinaciones de condiciones climáticas

Debido a que la lluvia es muy poco frecuente, **Apriori** no genera reglas útiles.
Se opta por un análisis de **combinaciones discretizadas** para estimar la probabilidad de lluvia por tipo de día.

1. **Discretización**:
   - `TEMP_MEAN` → Frío / Cálido
   - `HUM_MEAN` → Baja / Alta
   - `PNM_MEAN` → Baja / Alta
2. **Agrupación** por combinación y cálculo de:
   - Días totales
   - Días con lluvia
   - Porcentaje de lluvia


In [ ]:
# Discretización en 2 categorías
df_assoc = df_diario[['TEMP_MEAN','HUM_MEAN','PNM_MEAN','LLUEVE']].copy()
df_assoc['TEMP_CAT'] = pd.cut(df_assoc['TEMP_MEAN'], bins=[-5,20,50], labels=['Frio','Calido'])
df_assoc['HUM_CAT'] = pd.cut(df_assoc['HUM_MEAN'], bins=[0,85,100], labels=['Baja','Alta'])
df_assoc['PNM_CAT'] = pd.cut(df_assoc['PNM_MEAN'], bins=[950,1010,1050], labels=['Baja','Alta'])

# Agrupación
df_grouped = df_assoc.groupby(['TEMP_CAT','HUM_CAT','PNM_CAT']).agg(
    dias_total=('LLUEVE','count'),
    dias_lluvia=('LLUEVE','sum')
).reset_index()

df_grouped['porcentaje_lluvia'] = (df_grouped['dias_lluvia'] / df_grouped['dias_total'] * 100).round(2)

df_grouped.sort_values('porcentaje_lluvia', ascending=False)

## Clustering de días meteorológicos

Se aplica **K-Means** para agrupar días según sus condiciones meteorológicas:

- Variables: `TEMP_MEAN`, `HUM_MEAN`, `PNM_MEAN`, `WIND_SPEED_MEAN`
- Escalado previo para que todas tengan igual peso
- **PCA** para visualizar los clusters en 2D

Cada color representa un **tipo de día**:
- Cálido-Seco
- Frío-Húmedo
- Intermedio

In [ ]:
# Variables numéricas
X_clust = df_diario[['TEMP_MEAN','HUM_MEAN','PNM_MEAN','WIND_SPEED_MEAN']]

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clust)

# K-Means
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_diario['CLUSTER'] = kmeans.fit_predict(X_scaled)

# PCA para visualización
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=df_diario['CLUSTER'], cmap='viridis')
plt.title("Clusters de días meteorológicos (PCA 2D)")
plt.xlabel("Componente principal 1")
plt.ylabel("Componente principal 2")
plt.colorbar(label='Cluster')
plt.show()

# 🔹 Interpretación de componentes principales
pca_df = pd.DataFrame(pca.components_, columns=X_clust.columns, index=['PC1','PC2'])
print("Contribución de cada variable a los componentes principales:")
display(pca_df.T)

# Probabilidad de lluvia por cluster
df_diario.groupby('CLUSTER')['LLUEVE'].mean()

### Interpretación del PCA con Clustering

En el gráfico PCA 2D:

- **Eje X (Componente Principal 1)**: 
  - Gradiente entre **días cálidos y secos** (derecha) y **días fríos y húmedos** (izquierda).  
- **Eje Y (Componente Principal 2)**: 
  - Intensidad del **viento medio diario**.  
  - Arriba → días ventosos | Abajo → días calmos.

- **Colores de los puntos**: 
  - Cada color representa un **cluster identificado por K-Means**:  
    1. **Cluster 0 (amarillo/verde)** → días cálidos y secos  
    2. **Cluster 1 (violeta/azul)** → días fríos y húmedos  
    3. **Cluster 2 (verde/azulado)** → días intermedios

> Esta visualización permite entender **tipos de días** y su relación con la lluvia.


## Visualización con t-SNE

El **t-SNE (t-Distributed Stochastic Neighbor Embedding)** es una técnica de **reducción de dimensionalidad no lineal** utilizada principalmente para **visualización**.

- **Objetivo:** Representar datos de alta dimensión en 2D/3D preservando **vecindades locales**.
- **Ventaja:** Permite descubrir patrones o agrupamientos que **PCA lineal** podría no reflejar.
- **Limitación:** 
  - Los ejes no tienen interpretación física como PCA. 
  - Solo importa la **proximidad de puntos**: puntos cercanos → días meteorológicos similares.

In [ ]:
# t-SNE en 2D
tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000)
X_tsne = tsne.fit_transform(X_scaled)

# Visualización de clusters en t-SNE
plt.figure(figsize=(8,6))
plt.scatter(X_tsne[:,0], X_tsne[:,1], c=df_diario['CLUSTER'], cmap='viridis')
plt.title("Clusters de días meteorológicos (t-SNE 2D)")
plt.xlabel("t-SNE componente 1")
plt.ylabel("t-SNE componente 2")
plt.colorbar(label='Cluster')
plt.show()

### Interpretación del gráfico t-SNE

- Cada **punto** = un **día meteorológico**.
- **Colores** = clusters asignados por K-Means (tipos de días).  
- **Proximidad** = similitud de condiciones:  
  - Puntos cercanos → días con condiciones similares  
  - Puntos alejados → días con comportamientos distintos  
- **Los ejes no representan variables meteorológicas directas**, solo una proyección que facilita la visualización de grupos.

# Conclusión

En esta etapa se trabajó sobre los datos horarios procesados del SMN, aplicando técnicas de **minería de datos y análisis exploratorio** para descubrir patrones relacionados con la ocurrencia de lluvia.  
El objetivo principal fue **generar conocimiento a partir de los datos procesados**, preparando la información para etapas posteriores de clasificación y modelado predictivo.

1. **Generación de `LLUEVE`:**  
   - Se creó una etiqueta binaria que indica lluvia (1) o no lluvia (0), derivada de las condiciones meteorológicas.  
   - Se confirmó un **dataset desbalanceado**, con pocas ocurrencias de la clase positiva.

2. **Exploración y relaciones ocultas:**  
   - Los **heatmaps de correlación** mostraron que la lluvia está fuertemente asociada a **humedad alta** y **presión baja**.  
   - Al analizar por estación meteorológica, se observaron **patrones consistentes**.

3. **Combinaciones de condiciones:**  
   - Los días con **humedad alta y presión baja** alcanzan **máxima probabilidad de lluvia** en el histórico.  
   - Los **días cálidos y húmedos** son los más propensos a eventos de lluvia.

4. **Clustering y reducción de dimensionalidad:**  
   - El algoritmo **K-Means** identificó **3 tipos de días**: cálidos-secos, fríos-húmedos e intermedios.  
   - La **PCA 2D** permitió interpretar los ejes principales como:  
     - **PC1** → Gradiente cálido vs. frío-húmedo  
     - **PC2** → Intensidad del viento  
   - La técnica **t-SNE** confirmó la **separación de clusters** y reveló vecindades locales entre días con características similares.

Este análisis de **minería de datos** permitió comprender la estructura y los patrones del dataset, dejando la información **lista para la siguiente etapa de clasificación supervisada**, donde se evaluará la capacidad predictiva de distintos modelos sobre la variable `LLUEVE`.